In [5]:
# Import required libraries
import pandas as pd

# Read the CSV file
file_name = "../trades.csv"
df = pd.read_csv(file_name)

row_count = df.shape[0]
# drop header rows that appear multiple times in the csv file
df = df.drop(df[df.timestamp == "timestamp"].index)

restart_count = row_count - df.shape[0]

# timestamp,uuid,landed,accepted,rejected,errorType,errorContent,txn0Signature,txn1Signature,txn2Signature,arbSize,expectedProfit,hop1Dex,hop2Dex,sourceMint,intermediateMint,tipLamports,mempoolEnd,preSimEnd,simEnd,postSimEnd,calcArbEnd,buildBundleEnd,bundleSent

df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
df['mempoolEnd'] = pd.to_datetime(df['mempoolEnd'], unit='ms')
df['preSimEnd'] = pd.to_datetime(df['preSimEnd'], unit='ms')
df['simEnd'] = pd.to_datetime(df['simEnd'], unit='ms')
df['postSimEnd'] = pd.to_datetime(df['postSimEnd'], unit='ms')
df['calcArbEnd'] = pd.to_datetime(df['calcArbEnd'], unit='ms')
df['buildBundleEnd'] = pd.to_datetime(df['buildBundleEnd'], unit='ms')
df['bundleSent'] = pd.to_datetime(df['bundleSent'], unit='ms')
df[['landed', 'rejected']] = df[['landed', 'rejected']].fillna(
    False).astype('bool')
df[['accepted', 'arbSize', 'expectedProfit', 'tipLamports']] = df[[
    'accepted', 'arbSize', 'expectedProfit', 'tipLamports']].astype('float64')

# Display the first few records
print(df.head())
print(df.info())

                timestamp                                               uuid  \
0 2024-01-20 17:10:06.948  e8cbb417e4accd2b171387af49eb6b1925ca6b8962a561...   
1 2024-01-20 17:11:42.251  757157a7d0843efa854f7a0e8ea1e38337788ede43c577...   

   landed  accepted  rejected          errorType  \
0    True       0.0      True  simulationFailure   
1    True       0.0      True  simulationFailure   

                                        errorContent  \
0  {"txSignature":"4jv68kX7rqthGbLTZ7S4SWNVBngvLg...   
1  {"txSignature":"3korPFmEJz8TWcmdPvKQRdFtLFpSj9...   

                                       txn0Signature  \
0  5mrEi9myrg86EswqVP5HbLag5jkZA7TjU6BTNaupjXQVv7...   
1  3V8W2mP8dbjgwDZKYucy49XFcwRdTVq5fqMJDeZzDLVu8G...   

                                       txn1Signature       arbSize  ...  \
0  4jv68kX7rqthGbLTZ7S4SWNVBngvLgB7JTKQoVLwj5MJF2...  2.722061e+08  ...   
1  3korPFmEJz8TWcmdPvKQRdFtLFpSj962S8qpUHM9skCEfH...  2.015846e+09  ...   

                              intermed

In [2]:
!pip3 install pandas

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 80.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.6/346.6 KB 77.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 87.2 MB/s eta 0:00:0000:0100:01


In [14]:
print("Reading file: ", file_name)
print("First item timestamp (UTC): ", df['timestamp'].iloc[0])
print(f"Restart count: {restart_count}")

LAMPORTS_PER_SOL = 1000000000

# Calculate the percentage of rows where 'landed' is True compared to those with 'accepted' > 0
success_landed = df[(df['landed'] == True) & (df['accepted'] > 0)]
success_landed_count = success_landed.shape[0]
total_lamports_tipped = success_landed['tipLamports'].sum()
accepted_gt_0 = df[df['accepted'] > 0].shape[0]
if accepted_gt_0 > 0:
    percentage_landed_accepted = (success_landed_count / accepted_gt_0) * 100
else:
    percentage_landed_accepted = 0

# Calculate the percentage of rows with 'accepted' > 0 compared to the total
total_rows = df.shape[0]
percentage_accepted = (accepted_gt_0 / total_rows) * 100

# Print the results
print(f"Total sent: {total_rows}, accepted: {accepted_gt_0}, landed: {success_landed_count}")
print(
    f"Percentage landed of accepted: {percentage_landed_accepted:.2f}%")
print(
    f"Percentage accepted of sent: {percentage_accepted:.2f}%")
print(f"Total lamports tipped: {total_lamports_tipped} ({total_lamports_tipped / LAMPORTS_PER_SOL:.6f} SOL)")

# Find the error percentage for different error messages
errors_filtered = df[df['errorType'].notnull()]
sim_failures = errors_filtered[errors_filtered['errorType'].str.contains(
    "simulationFailure")]
total_errors = errors_filtered.shape[0]

sim_errors_my_fault = sim_failures[sim_failures.apply(
    lambda row: row['txn1Signature'] in row['errorContent'] or row['txn2Signature'] in row['errorContent'], axis=1)]
sim_errors_backrun_txn = sim_failures[sim_failures.apply(
    lambda row: row['txn0Signature'] in row['errorContent'], axis=1)]
sim_tx_already_processed = sim_failures[sim_failures['errorContent'].str.contains(
    "This transaction has already been processed")]
sim_other = sim_failures[~sim_failures.index.isin(
    sim_errors_my_fault.index.union(
        sim_errors_backrun_txn.index).union(sim_tx_already_processed.index))]

# Group non-simulation errors by errorType
non_sim_errors = errors_filtered[~errors_filtered['errorType'].str.contains("simulationFailure")]
sending_errors = errors_filtered[errors_filtered['errorType'].str.contains("sendingError")]

error_groups = non_sim_errors.groupby("errorType").size().reset_index(name='count')

# Calculate the error totals and percentages for sim errors and other errors
error_totals = {
    'sim_errors_my_fault': sim_errors_my_fault.shape[0],
    'sim_errors_backrun_txn': sim_errors_backrun_txn.shape[0],
    'sim_tx_already_processed': sim_tx_already_processed.shape[0],
    'sim_other': sim_other.shape[0],
}

error_percentages = {k: (v / total_errors) * 100 for k, v in error_totals.items()}
for _, row in error_groups.iterrows():
    error_name = f"{row['errorType']}"
    error_totals[error_name] = row['count']
    error_percentages[error_name] = (row['count'] / total_errors) * 100

# Remove errors with zero count
error_totals = {k: v for k, v in error_totals.items() if v > 0}
error_percentages = {k: v for k, v in error_percentages.items() if k in error_totals}
print(sim_errors_my_fault)
print(sim_errors_backrun_txn)
print(sim_tx_already_processed)
print(sim_other)
# Print the error totals and their percentages
print("\nError Totals and Percentages:")
for error_type, total in error_totals.items():
    percentage = error_percentages[error_type]
    print(f"{error_type}: Total: {total}, Percentage: {percentage:.2f}%")

Reading file:  ../trades.csv
First item timestamp (UTC):  2024-01-20 17:10:06.948000
Restart count: 0
Total sent: 2, accepted: 0, landed: 0
Percentage landed of accepted: 0.00%
Percentage accepted of sent: 0.00%
Total lamports tipped: 0.0 (0.000000 SOL)
                timestamp                                               uuid  \
0 2024-01-20 17:10:06.948  e8cbb417e4accd2b171387af49eb6b1925ca6b8962a561...   
1 2024-01-20 17:11:42.251  757157a7d0843efa854f7a0e8ea1e38337788ede43c577...   

   landed  accepted  rejected          errorType  \
0    True       0.0      True  simulationFailure   
1    True       0.0      True  simulationFailure   

                                        errorContent  \
0  {"txSignature":"4jv68kX7rqthGbLTZ7S4SWNVBngvLg...   
1  {"txSignature":"3korPFmEJz8TWcmdPvKQRdFtLFpSj9...   

                                       txn0Signature  \
0  5mrEi9myrg86EswqVP5HbLag5jkZA7TjU6BTNaupjXQVv7...   
1  3V8W2mP8dbjgwDZKYucy49XFcwRdTVq5fqMJDeZzDLVu8G...   

          

In [12]:
# 0x1771 is Jupiter SlippageToleranceExceeded

print(sending_errors.to_string())

Empty DataFrame
Columns: [timestamp, uuid, landed, accepted, rejected, errorType, errorContent, txn0Signature, txn1Signature, arbSize, expectedProfit, hop1Dex, hop2Dex, hop3Dex, sourceMint, intermediateMint1, intermediateMint2, tipLamports, mempoolEnd, preSimEnd, simEnd, postSimEnd, calcArbEnd, buildBundleEnd, bundleSent]
Index: []
